In [1]:

import io
import os
import PySimpleGUI as sg
from PIL import Image
import cv2


file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]

def main():
    layout = [
        [
            sg.Text("Image File"),
            sg.Input(size=(25, 1), key="-FILE-"),
            sg.FileBrowse(file_types=file_types),
            sg.Button("Run"),
        ],
        [sg.Image(key="-IMAGE-")],
    ]

    window = sg.Window("Object Detection GUI- Yolo-V4", layout)

    while True:
        event, values = window.read()
        if event == "Exit" or event == sg.WIN_CLOSED:
            break
        if event == "Run":
            filename = values["-FILE-"]
            if os.path.exists(filename):

                image = cv2.imread(values["-FILE-"])
                Conf_threshold = 0.5
                NMS_threshold = 0.7
                COLORS = [(0, 255, 0), (0, 0, 255), (255, 0, 0),
                          (255, 255, 0), (255, 0, 255), (0, 255, 255)]

                class_name = []
                with open('classes.txt', 'r') as f:
                    class_name = [cname.strip() for cname in f.readlines()]
                # print(class_name)
                net = cv2.dnn.readNet('yolov4-tiny.weights', 'yolov4-tiny.cfg')
                net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

                model = cv2.dnn_DetectionModel(net)
                model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

                classes, scores, boxes = model.detect(image, Conf_threshold, NMS_threshold)
                for (classid, score, box) in zip(classes, scores, boxes):
                    color = COLORS[int(classid) % len(COLORS)]
                    label = "%s : %f" % (class_name[classid[0]], score)
                    cv2.rectangle(image, box, color, 2)
                    cv2.putText(image, label, (box[0], box[1]-10),cv2.FONT_HERSHEY_COMPLEX, 0.3, color, 2)

                cv2.imwrite("detected_image.png",image)
                
                image = Image.open("detected_image.png") # grab the image
                image.thumbnail((600, 600)) # resize the image
                bio = io.BytesIO() # convert the image into io.BytesIO to be later displayed and save it in memory
                image.save(bio, format="PNG")
                window["-IMAGE-"].update(data=bio.getvalue())

    window.close()


main()